In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Import các thư viện thông dụng

In [35]:
file_path = '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'
train_set = pd.read_csv(file_path)
predict_path = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
predict_set = pd.read_csv(predict_path)

Đọc dữ liệu cho 2 tập train và predict

In [36]:
train_set.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
predict_set.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**B1: Xử lý dữ liệu trước khi đưa vào mô hình**


In [38]:
train_set.shape

(3960, 82)

In [39]:
train_set.isnull().sum()

id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                               1405
                                          ... 
SDS-SDS_Total_Raw                         1351
SDS-SDS_Total_T                           1354
PreInt_EduHx-Season                        420
PreInt_EduHx-computerinternet_hoursday     659
sii                                       1224
Length: 82, dtype: int64

In [40]:
# Loại bỏ các hàng có output(sii) là NaN không có giá trị trong việc dự đoán
train_set = train_set.dropna(subset = 'sii')

In [41]:
# Loại bỏ các giá trị NaN còn lại bằng giá trị mode của cột đó
for col in train_set:
    train_set[col] = train_set[col].fillna(train_set[col].mode()[0])
for col in predict_set:
    predict_set[col] = predict_set[col].fillna(predict_set[col].mode()[0])

In [42]:
# Xử lý dữ liệu không hợp lệ, noise
# Bỏ qua

In [43]:
# Chia tập train ra thành X_train và y_train
# Cắt bớt cột id của tập train để đưa vào mô hình
X_train = train_set.drop(['id','sii'], axis = 1).copy()
y_train = train_set['sii'].astype('int64').copy()

In [44]:
# Cắt bớt cột id của tập test để đưa vào mô hình dự đoán
X_predict = predict_set.drop('id',axis = 1).copy()

In [45]:
# Trong tập test, số lượng feature ít hơn tập train
# Có thể xóa bớt feature trong tập train hoặc thêm feature vào tập test
# Ở đây thêm vào feature cho đủ
for col in X_train:
    if col not in X_predict.columns:
        X_predict[col] = np.nan
        # Gán các giá trị cho cột feature bằng mode của tập train
        X_predict[col] = X_predict[col].fillna(X_train[col].mode())

# Sắp xếp lại tập test cho giống với tập train
X_predict = X_predict[X_train.columns]


In [46]:
X_train.shape

(2736, 80)

In [47]:
X_predict.shape

(20, 80)

In [48]:
y_train.shape

(2736,)

In [49]:
y_train.dtype

dtype('int64')

**B2: Mã hóa dữ liệu**

Ta phải mã hóa 2 tập X_train và X_predict để có thể đưa vào mô hình có sẵn

In [50]:
# Mã hóa dữ liệu
from sklearn.preprocessing import LabelEncoder

In [51]:
# 1 ma trận sử dụng để map giá trị X_predict và X_train
encoders = {}
#X_train
for column in X_train:
    encoder = LabelEncoder()
    X_train[column] = encoder.fit_transform(X_train[column])
    encoders[column] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
#X_predict
for column in X_predict:
    if column in encoders:
        X_predict[column] = X_predict[column].map(encoders[column]).fillna(X_train[column].mode()[0]).astype('int64')

In [52]:
X_train.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,0,0,0,3,17,0,829,33,82,4,...,4,4,2,4,55,1,12,9,0,3
1,2,4,0,1,26,0,102,50,58,3,...,0,0,0,0,0,0,23,20,2,0
2,2,5,1,0,37,0,759,148,215,4,...,2,2,1,1,28,0,15,12,2,2
3,3,4,0,0,37,2,1161,143,247,4,...,4,3,4,1,44,2,8,5,3,0
5,1,8,1,3,16,2,1766,172,404,4,...,3,1,2,1,34,2,17,14,1,0


In [53]:
X_predict.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,0,0,0,3,17,0,829,33,82,5,...,0,0,0,0,0,0,4,1,0,3
1,2,4,0,2,17,0,102,50,58,3,...,0,0,0,0,0,0,23,20,2,0
2,2,5,1,0,37,0,759,148,215,5,...,0,0,0,0,0,0,15,12,2,2
3,3,4,0,0,37,2,1161,143,247,5,...,0,0,0,0,0,2,8,5,3,0
4,1,13,1,2,17,0,102,132,58,5,...,0,0,0,0,0,0,4,1,0,2


**B3: Lựa chọn mô hình và huấn luyện**

Với bài toán multiclassification, các mô hình có thể lựa chọn là:


* Logistic Regression (Multinomial)
* Decision Trees
* Random Forests
* k-Nearest Neighbors (k-NN)
* SVM
* Naive-Bayes

Cùng với đó là các mô hình nâng cao:
* XGboost: Kết hợp nhiều mô hình yếu để tạo ra 1 mô hình mạnh hơn, đây là thuật toán thường được sử dụng trong các competition nhờ độ hữu ích của nó
* Neural network: MLP hoặc mô hình deep learning có sẵn của thư viện tensorflow hoặc Pytorch

Ở đây em sẽ thử nghiệm 2 mô hình XGBoost và mạng NN của tensorflow

In [54]:
# Import thư viện mô hình
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Chia tập train thành tập huấn luyện và kiểm tra

In [55]:
encoder = LabelEncoder()
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [57]:
# Chuẩn hóa dữ liệu đầu vào
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
# Model init
model = models.Sequential()
# Input layer
model.add(layers.InputLayer(shape=(X_train.shape[1],)))
# Hidden layers
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(256,activation = 'relu'))
model.add(layers.Dense(128,activation = 'relu'))
model.add(layers.Dense(32, activation='relu'))
# Output layer
model.add(layers.Dense(4, activation='softmax'))

In [59]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',  # Hàm mất mát cho phân loại nhiều lớp
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6522 - loss: 0.7519 - val_accuracy: 0.9015 - val_loss: 0.2561
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9241 - loss: 0.2141 - val_accuracy: 0.9215 - val_loss: 0.2092
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9522 - loss: 0.1459 - val_accuracy: 0.9398 - val_loss: 0.1497
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9742 - loss: 0.0707 - val_accuracy: 0.9507 - val_loss: 0.1217
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9854 - loss: 0.0557 - val_accuracy: 0.9453 - val_loss: 0.1275
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9901 - loss: 0.0368 - val_accuracy: 0.9434 - val_loss: 0.1434
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9879 - loss: 0.0378 - val_accuracy: 0.9252 - val_loss: 0.1758
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9910 - loss: 0.0304 - val_accuracy: 0.9471 - v

In [60]:
# Đánh giá mô hình
# Bỏ qua

**B4: Dự đoán kết quả**

In [70]:
# Dự đoán
predict = model.predict(X_predict)
predict = np.argmax(predict,axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [71]:
predict

array([0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1])

**In ra file output**

In [63]:
# In ra file output
import os
file_path = '/kaggle/working/submission.csv'
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"File {file_path} đã được xóa.")
else:
    print(f"File {file_path} không tồn tại.")

File /kaggle/working/submission.csv đã được xóa.


In [64]:
pf_pred = pd.DataFrame(columns=['id', 'sii'])
pf_pred['id'] = predict_set['id']
pf_pred['sii'] = predict
pf_pred.head()
# set an index to avoid a 'blank' row number column and write to a file named 'submission.csv'
pf_pred.to_csv('submission.csv',index=False)

In [65]:
# Kiểm tra dữ liệu file output
file_p = '/kaggle/working/submission.csv'
output = pd.read_csv(file_p)
output.head()

,id,sii
0,00008ff9,0
1,000fd460,0
2,00105258,1
3,00115b9f,0
4,0016bb22,1


In [66]:
# Các cách cải tiến dự đoán:
# Feature selection
# Tối ưu hóa tham số
# Thử nghiệm các mô hình khác
# Kết hợp nhiều mô hình, XGBoost